# 한글-Claude-v2 Model: Invoke Bedrock model for text generation using zero-shot prompt
---

### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

#### 사용 사례
Amazon Bedrock에서 모델의 생성 기능을 시연하기 위해 이메일 생성 사용 사례를 살펴보겠습니다.

#### 페르소나
귀하는 AnyCompany의 고객 서비스 관리자인 권율이며 일부 고객은 고객 서비스에 만족하지 않고 고객 지원 엔지니어가 제공하는 서비스에 대해 부정적인 피드백을 제공하고 있습니다. 이제는 열악한 서비스에 대해 겸허히 사과하는 고객에게 응답하고 신뢰를 회복하고 싶습니다. 인간 친화적이고 이전 이메일 서신에서 고객의 정서에 맞게 개인화된 대량의 이메일을 생성하려면 LLM의 도움이 필요합니다.

#### 구현
이 사용 사례를 이행하기 위해 이 노트북에서는 고객의 이전 이메일을 기반으로 감사 메모가 포함된 이메일을 생성하는 방법을 보여줍니다. Boto3 클라이언트와 함께 Amazon Bedrock API를 사용하는 Amazon Titan Text Large 모델을 사용합니다.

In [2]:
! pip list | grep langchain
! pip list | grep opensearch

langchain                            0.0.312
opensearch-py                        2.3.2


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## Anthroppic Claude models

인공 지능 클로드 모델은 응답의 무작위성과 다양성을 제어하기 위해 [다음 매개 변수를 지원](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html)합니다.

Temperature (temperature)- 응답의 무작위성을 줄이려면 낮은 값을 사용합니다.

Top P(topP) - 낮은 값을 사용하여 가능성이 낮은 옵션을 무시합니다.

Top K(topK) - 모델이 다음 토큰을 생성하는 데 사용하는 토큰 선택의 수를 지정합니다.

<pre>
{
    "prompt": "\n\nHuman:<prompt>\n\nAssistant:",
    "temperature": float,
    "top_p": float,
    "top_k": int,
    "max_tokens_to_sample": int,
    "stop_sequences": ["\n\nHuman:"]
}
</pre>

## boto3_bedrock 사용
boto3 bedrock-runtime을 이용하여 claude 모델에 문의해봅니다. 

temperature 값을 높은값으로 사용해봅니다.

In [5]:
import boto3
import json

modelId = 'anthropic.claude-v2'
accept = 'application/json'
contentType = 'application/json'

# NOTE: Body must same as following stylea
prompt = "Hello"
claude_prompt = f"\n\nHuman:{prompt}\n\nAssistant:"
body = json.dumps({
                "prompt": claude_prompt,
                "temperature": 0.1,
                "top_p": 1,
                "top_k": 250,
                "max_tokens_to_sample": 200,
                "stop_sequences": ["\n\nHuman:"]
                })
response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Hello! Nice to meet you.


## langchain with Bedrock을 이용

langchain with Bedrock 한글로 질의해봅니다.

In [6]:
prompt = "삼성전자의 2022년 매출을 얼마인가요?"
claude_prompt = f"\n\nHuman:{prompt}\n\nAssistant:"

In [7]:
from langchain.llms import Bedrock

textgen_llm = Bedrock(
    # model_id='anthropic.claude-v2',
    model_id=bedrock_info.get_model_id(model_name='Claude-V2'),
    client=boto3_bedrock,
    model_kwargs={
        "prompt": claude_prompt,
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.999,
    } 
)
textgen_llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f64f33398d0>, model_id='anthropic.claude-v2', model_kwargs={'prompt': '\n\nHuman:삼성전자의 2022년 매출을 얼마인가요?\n\nAssistant:', 'max_tokens_to_sample': 512, 'stop_sequences': ['\n\nHuman:'], 'temperature': 0, 'top_p': 0.999})

In [8]:
response = textgen_llm(claude_prompt)

print(response)

## langchain with Bedrock을 Streaming 방식

langchain with Bedrock 한글로 질의해봅니다.

In [11]:
prompt = "삼성전자의 Knox 어카운트 무엇인가요?"
claude_prompt = f"\n\nHuman:{prompt}\n\nAssistant:"

In [12]:
from langchain.llms import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

textgen_llm = Bedrock(
    # model_id='anthropic.claude-v2',
    model_id=bedrock_info.get_model_id(model_name='Claude-V2'),
    client=boto3_bedrock,
    model_kwargs={
        "prompt": claude_prompt,
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.999,
    }, 
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)
textgen_llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f64f33398d0>, model_id='anthropic.claude-v2', model_kwargs={'prompt': '\n\nHuman:삼성전자의 Knox 어카운트 무엇인가요?\n\nAssistant:', 'max_tokens_to_sample': 512, 'stop_sequences': ['\n\nHuman:'], 'temperature': 0, 'top_p': 0.999}, streaming=True, callbacks=[<langchain.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f64f333b100>])

In [13]:
num_tokens = textgen_llm.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Our prompt has 42 tokens


In [14]:
response = textgen_llm(claude_prompt)

 삼성전자의 Knox는 모바일 기기의 보안과 관리를 위한 플랫폼입니다. 

Knox의 주요 기능은 다음과 같습니다:

- 개인용과 업무용을 분리하는 이중화: 개인 앱과 데이터를 업무 앱과 데이터에서 별도로 관리할 수 있습니다.

- 보안 강화: 기기의 보안을 강화하여 해킹과 악성 앱을 방지합니다. 

- 기기 관리: 기업은 Knox를 통해 업무 기기를 원격으로 관리할 수 있습니다.

- 앱 보안: 업무 앱은 Knox 내에서 실행되어 추가 보안이 적용됩니다.

Knox 어카운트는 이러한 Knox 서비스를 사용하기 위해 필요한 삼성 계정입니다. Knox 기능을 활성화하려면 삼성 계정에 Knox 라이선스를 등록해야 합니다.

In [15]:
print(response)

 삼성전자의 Knox는 모바일 기기의 보안과 관리를 위한 플랫폼입니다. 

Knox의 주요 기능은 다음과 같습니다:

- 개인용과 업무용을 분리하는 이중화: 개인 앱과 데이터를 업무 앱과 데이터에서 별도로 관리할 수 있습니다.

- 보안 강화: 기기의 보안을 강화하여 해킹과 악성 앱을 방지합니다. 

- 기기 관리: 기업은 Knox를 통해 업무 기기를 원격으로 관리할 수 있습니다.

- 앱 보안: 업무 앱은 Knox 내에서 실행되어 추가 보안이 적용됩니다.

Knox 어카운트는 이러한 Knox 서비스를 사용하기 위해 필요한 삼성 계정입니다. Knox 기능을 활성화하려면 삼성 계정에 Knox 라이선스를 등록해야 합니다.
